In [1]:
from pyspark import SparkContext
sc = SparkContext("local[*]","PySpark Tutorial")

In [2]:
import json
from operator import add
file = 'data/Gamergate.json'
tweets_file = 'data/tweets'

In [3]:
tweets = []
for line in open(file, 'r'):
    tweets.append(json.loads(line))

part 1

In [4]:
import numpy as np
import statistics

In [14]:
followers_map = {}
id_map = {}
day_map = {}
rts = []
for t in tweets:
    twitter_id = t['user']['id']
    screen_name = t['user']['screen_name']
    followers = t['user']['followers_count']
    day = t['created_at'][:3]
    
    rts.append(t['retweet_count'])
    followers_map[screen_name] = followers
    
    if twitter_id in id_map:
        id_map[twitter_id]+=1
    else:
        id_map[twitter_id]=1
        
    if day in day_map:
        day_map[day] += 1
    else:
        day_map[day] = 1
        

In [40]:
print('Total number of unique twitter accounts is', format(len(id_map)))

Total number of unique twitter accounts is 21049


In [41]:
sortedmap = sorted(followers_map.items(), key=lambda x: x[1], reverse=True)
print('Top 3 users are', format(sortedmap[:3]))

Top 3 users are [('washingtonpost', 12584968), ('TheOnion', 11427200), ('WIRED', 10587728)]


In [42]:
day_map

{'Tue': 44558, 'Wed': 43454, 'Thu': 16902}

In [51]:
statistics.stdev(rts)

320.7978465773098

In [6]:
lines = sc.textFile(file)

In [4]:
lineJSONS = lines.map(lambda s: (json.loads(s)))

In [20]:
lineJSONS.take(1)

[{'retweet_count': 338,
  'user': {'description': 'Seattle-based amateur chef and gamer with deep passion for theater and music.',
   'followers_count': 582,
   'default_profile_image': False,
   'profile_image_url_https': 'https://pbs.twimg.com/profile_images/977029984966533120/94BsR1Ae_normal.jpg',
   'profile_link_color': 'FF0000',
   'created_at': 'Thu Feb 12 19:27:48 +0000 2009',
   'profile_use_background_image': True,
   'time_zone': 'Pacific Time (US & Canada)',
   'has_extended_profile': True,
   'following': False,
   'translator_type': 'none',
   'profile_sidebar_border_color': '65B0DA',
   'follow_request_sent': False,
   'profile_background_tile': True,
   'id': 20708129,
   'utc_offset': -25200,
   'profile_text_color': '3D1957',
   'location': 'Seattle, WA',
   'lang': 'en',
   'listed_count': 54,
   'friends_count': 414,
   'profile_sidebar_fill_color': '7AC3EE',
   'profile_background_color': '642D8B',
   'protected': False,
   'profile_image_url': 'http://pbs.twimg.co

In [5]:
userJSONS = lineJSONS.map(lambda x: x['user'])

In [6]:
users = lineJSONS.map(lambda x: x['user']['id'])

In [8]:
users.distinct().count()

21049

In [16]:
created = lineJSONS.map(lambda x: x['created_at'])

In [19]:
created.count()

104914

In [14]:
follows = userJSONS.map(lambda x: (x['screen_name'], x['followers_count']))

In [22]:
top3 = follows.sortBy(lambda x: x[1], ascending=False).collect()[:3]

In [23]:
top3

[('washingtonpost', 12584968), ('TheOnion', 11427200), ('WIRED', 10587728)]

In [53]:
follows[0]['followers_count']

12584968

In [61]:
tuesdays = lineJSONS.filter(lambda x: x['created_at'][:3] == 'Tue')

In [62]:
tuesdays.count()

44558

In [74]:
lineJSONS.take(1)[0]['text']

'RT @misterbrilliant: First scene from the new series of Firefly has been leaked. #GamerGate http://t.co/5D3OMkwOfd'

part 2

In [64]:
rts = lineJSONS.map(lambda x: x['retweet_count'])

In [66]:
rts.mean()

114.46304592332771

In [67]:
rts.max()

7181

In [69]:
rts.stdev()

320.7963177126635

part 3

In [7]:
ts = lines.flatMap(lambda x: x.split(' '))
ks = ts.map(lambda x: (x,1)).reduceByKey(add)

In [8]:
ks.take(1)

[('{"description":', 334694)]

In [9]:
m = ks.filter(lambda x: x[0] == 'mindless').collect()

In [10]:
m

[('mindless', 262)]

In [6]:
tweetlines = sc.textFile(tweets_file)

In [7]:
tweetlines.count()

21441

In [8]:
tweets = tweetlines.flatMap(lambda x: x.split(' '))
keys = tweets.map(lambda x: (x,1)).reduceByKey(add)


In [9]:
freq = keys.map(lambda x: (x[1], x[0])).sortByKey(False).take(1)

In [10]:
freq

[(149569, '#gamergate')]

In [31]:
freq[0][1]

'#gamergate'

In [178]:
mindless = keys.filter(lambda x: x[0] == 'mindless').collect()

In [179]:
mindless

[('mindless', 206)]

In [180]:
tweets.take(2)

['|********************', 'rt']

In [68]:
chunks = keys.filter(lambda x: x[0] == '|********************'  or x[0] == '********************|')

In [70]:
chunks.take(2)

[('|********************', 21441), ('********************|', 21441)]

In [185]:
keys

PythonRDD[169] at RDD at PythonRDD.scala:53

In [186]:
keys.take(3)

[('|********************', 21441), ('scene', 88), ('new', 3666)]